# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.0/229.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "tokyotech-llm/Swallow-13b-instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{input}\n\n### 応答:"
    )
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        temperature=0.99,
        top_p=0.95,
        do_sample=True,
        #pad_token_id=tokenizer.pad_token_id,
        #bos_token_id=tokenizer.bos_token_id,
        #eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            writer.write({"pred": generated_text})

            print("==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print()

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 仕事の熱意を取り戻す5つの方法

1.仕事を好きになる
2.目標を設定する
3.ワークアウト
4.セルフケアを行う
5.感謝をしてください

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. 「クマの冒険」。

クマはいつも海が何であるかを疑問に思っていました。彼はそれがどれほど壮大であるかを想像し、彼の興味をそそりました。彼はその広大さについて考え、そこに何があるのかと思いました。ある日、彼は岩の上に座って海を眺めていました。遠くに、水の大きな塊が見えました。彼はそれがとても速く動いているのを見て、それに興味をそそられました。

それで、クマは彼の冒険を始めるために泳ぐことにしました。彼は最初は怖かったが、すぐに夢中になり、それがどれほど美しくて楽しいかに驚いた。彼は海の下の不思議な生き物と会い、彼がどれほど孤独であるかについて泣きました。彼はついにアザラシと友達になりました - 彼が彼の孤独について話すと、アザラシはクマが彼が望むなら彼と一緒に住むことができると言いました。しかし、クマは海の外の世界で家に帰りたかったので、彼らは友達になりました。

海を離れて、クマは彼がどのように感じていたかを考えました。彼は冒険について考え、新しい友達と出会い、海の下の世界を探検するのがどれほどエキサイティングであるかに気づきました。彼はアザラシと一緒に、彼らの友達であり続けることを計画しました。そして、クマは彼の冒険が彼が思っていたよりずっとエキサイティングだったことを知っていました。

Q. 古代ギリシャを学ぶ上で知っておくべきポイントは？
A. 古代ギリシャを知るための5つのポイント

1.ギリシャは歴史的、文化的、社会的、政治的に重要な役割を担った大国であった。

2.ギリシャ語のアルファベットは、2,500年以上にわたって文字、数学、科学、芸術、哲学の進歩を促進する役割を果たした。

3.哲学、数学、科学、政治、文学など、あらゆる分野において、古代ギリシャは重要な役割を果たした。

4.ギリシャは、近代においても西洋思想の形成に影響を与えた重要な社会的および文化的中心地であり、今日でも深い影響を与えている。

5.古代ギリシャは、その